In [1]:
from os import chdir
chdir('/home/jovyan')

In [2]:
from lib.postgres import connect_to_postgres

# Attribute type

age: continuous \
workclass: class \
education: class \
education-num: continuous \
marital-status: class \
relationship: class \
race: class \
sex: class \
capital-gain: continuous \
capital-loss: continuous \
hours-per-week: continuous \
native-country: class \
income_labe: class

In [4]:
con, cur = connect_to_postgres()
cur.execute("""
BEGIN;
ALTER TABLE adult ADD COLUMN _id SERIAL PRIMARY KEY;
ALTER TABLE adult ADD COLUMN target BOOLEAN;
COMMIT;
""")
con.close()

In [22]:
con, cur = connect_to_postgres()
cur.execute("""SELECT distinct(income_label) FROM adult;""") 
print(cur.fetchall())
con.close()

[(' <=50K',), (' >50K',)]


In [23]:
con, cur = connect_to_postgres()
cur.execute("""SELECT _id, income_label FROM adult WHERE target IS NULL;""")
this_id, income_label = cur.fetchone()

greater_than_50k = (income_label == ' >50K')

cur.execute("""
BEGIN;
UPDATE adult
SET target = {}
WHERE _id = {};
COMMIT;
""".format(greater_than_50k, this_id))

con.close()

In [24]:
con, cur = connect_to_postgres()
cur.execute("""
SELECT _id, income_label, target
FROM adult WHERE _id = {};
""".format(this_id))
print(this_id, cur.fetchone())
con.close()

1 (1, ' <=50K', False)


In [25]:
def encode_target(_id):
    """Encode the target for a single row as a boolean value. Takes a row _id."""
    con, cur = connect_to_postgres()
    cur.execute("""SELECT _id, income_label FROM adult where _id ={}""".format(_id))
    this_id, income_label = cur.fetchone()
    assert this_id == _id
    greater_than_50k = (income_label == ' >50K')
    cur.execute("""
        BEGIN;
        UPDATE adult
        SET target = {}
        WHERE _id = {};
        COMMIT;
        """.format(greater_than_50k, _id))
    con.close()

In [26]:
con, cur = connect_to_postgres()
cur.execute("""SELECT _id FROM adult WHERE target IS NULL;""")
this_id, = cur.fetchone()
encode_target(this_id)
con.close()

In [27]:
con, cur = connect_to_postgres()
cur.execute("""
    SELECT _id, income_label, target
    FROM adult WHERE _id = {};
""".format(this_id))
print(this_id, cur.fetchone())
con.close()

2 (2, ' <=50K', False)
